# [CPSC 310](https://github.com/GonzagaCPSC310) Data Mining
[Gonzaga University](https://www.gonzaga.edu/)

[Gina Sprint](http://cs.gonzaga.edu/faculty/sprint/)

# Attribute Selection
What are our learning objectives for this lesson?
* Use entropy to select attributes to split on
* Create decision trees with continuous attributes

Content used in this lesson is based upon information in the following sources:
* Dr. Shawn Bowers' Data Mining notes

## Selecting Attributes
Resulting decision tree depends on attribute selection approach
* Want high predictive accuracy with small number of rules
* In practice, using **"information gain"** does well (popular approach)

Basic idea:
* Select attribute with the largest "information gain"
* Typically the attribute that most "unevenly" partitions the instances on class

How it works: use "Shannon Entropy" as a measure of information content
* Q: how many bits are needed to represent numbers between 1 and 64?
    * $log_2 64 = 6$ bits
* What if instead we had messages involving combinations of 64 words
    * Could capture each word using a 6 bit number
    * Thus a message with 10 words would cost 10 × 6 = 60 bits
* However, if we knew more about the distribution of words, we could (on average) use fewer bits per message!
    * e.g., "the" occurs more than any other word
    * Use encodings whose lengths are inversely proportional to their frequencies (probabilities)
* Entropy gives a precise lower bound for expected number of bits needed to encode a message (based on a probability distribution)

## Entropy $E$
The details:
* Entropy $E = 0$ implies low content (e.g., all values are the same)
* Highest entropy value when all values equally likely (most content)
* Entropy formula assumes information encoded in bits ... (thus, $log_2$)
$$E = -\sum_{i=1}^{n}p_i log_2(p_i)$$
    * $n$ for us is the number of class labels
    * $p_i$ is proportion of instances labeled with i (i.e., $P(C)$ for $C = i$)
    * Note $p_i$ is assumed to be strictly greater than 0, up to and including 1
* What the formula is saying:
    * Since $0 < p_i \leq 1$, we know that $-p_i log_2(p_i) \geq 0$ is positive
    * e.g., for $log_2(0.5) = y$, we have $2^y = \frac{1}{2}$, which means $y = -1$
    * If $p_i = 1$, then $-p_i log_2(p_i) = 0$
    * $E$ has the highest value when labels are equally distributed
    
The function $-p_i log_2(p_i)$ for $0 < p_i \leq 1$

<img src="https://raw.githubusercontent.com/GonzagaCPSC310/U5-Decision-Trees/master/figures/entropy_graph.png" width="500"/>

The function $-(p_i log_2(p_i) + (1-p_i) log_2 (1 - p_i))$

<img src="https://raw.githubusercontent.com/GonzagaCPSC310/U5-Decision-Trees/master/figures/entropy_binary_classification_graph.png" width="500"/>

* This mimics having just two labels
* As shown, the closer the two labels the higher the entropy

Pick the attribute that maximizes information gain
* Information Gain = $E_{start} - E_{new}$
    * At each partition, pick attribute with highest information gain
    * That is, split on attribute with greatest reduction in entropy
    * Which means find attribute with smallest $E_{new}$

### Example Computing $E$
Assume we have: $p_{yes} = 3/8$ and $p_{no} = 5/8$

Q: What is $E$ for this distribution? ... in Python:

In [2]:
import math

p_yes = 3 / 8
p_no = 5 / 8
E = -(p_yes * math.log(p_yes, 2) + p_no * math.log(p_no, 2))
print(E)

0.9544340029249649


Now assume we have: $p_{yes} = 2/8$ and $p_{no} = 6/8$

Q: Will $E$ for this distribution be larger or smaller? ... should be smaller!

In [3]:
p_yes = 2 / 8
p_no = 6 / 8
E = -(p_yes * math.log(p_yes, 2) + p_no * math.log(p_no, 2))
print(E)

0.8112781244591328


### Calculating $E_{new}$ for an Attribute
Assume we want to partition $D$ on an attribute $A$
* Where $A$ has values $a_1, a_2, ... , a_v$
* Creating partitions $D_1, D_2, ... , D_v$

We'd like each partition to be "pure" (contain instances of same class label)
* They may not be, however (i.e., they may have "clashes")
* The amount of additional info still needed for a "pure" classification is:
$$E_{new} = \sum_{j=1}^{v} \frac{|D_j|}{|D|} \times E_{D_j}$$

* where $E_{D_j}$ is the entropy of partition $D_j$
* $\frac{|D_j|}{|D|}$ is the likelihood an instance is in the $j$-th partition
* Thus, $E_{new}$ is a **weighted average** of the attributes corresponding partitions

We pick the attribute with the smallest $E_{new}$ value
* This means the smallest amount of information needed to classify an instance

## Lab Tasks

|standing |job_status |credit_rating |buys_iphone|
|-|-|-|-|
|1 |3 |fair |no|
|1 |3 |excellent |no|
|2 |3 |fair |yes|
|2 |2 |fair |yes|
|2 |1 |fair |yes|
|2 |1 |excellent |no|
|2 |1 |excellent |yes|
|1 |2 |fair |no|
|1 |1 |fair |yes|
|2 |2 |fair |yes|
|1 |2 |excellent |yes|
|2 |2 |excellent |yes|
|2 |3 |fair |yes|
|2 |2 |excellent |no|
|2 |3 |fair |yes|

1. Calculate $E_{new}$ for job_status for whole table
1. Calculate $E_{new}$ for credit_rating for whole table
1. What is the general $E_{new}$ algorithm?

## Dealing with Continuous Attributes
* So far we've mainly assumed categorical attributes
* Like always we can discretize continuous attributes first
* Alternatively, we can use Entropy to find a "split point"

The random approach:
* Pick a random value $v$ from the set of values in the attribute
* Use $v$ as a "split point"
* i.e., divide into two partitions: $\leq v$ and $> v$

Using entropy instead:
1. Sort the values in ascending order $[v_1, v_2, ... , v_k]$
1. For each split point $v$ in $v_1$ through $v_{k−1}$ calculate $E_{new}$
    * Thus, two partitions in each case ($\leq v$ and $> v$)
    * Alternatively, use half-way points between adjacent values $v_i$ and $v_{i+1}$
1. Select the split point that minimizes $E_{new}$